**mlops zoomcamp 2023 by DataTalksClub**

**homework 1**

This notebook is for the first homework of the mlops course. The objective are to explore the data and to train a model to predict the duration of a ride.

Data is from [NYC taxi dataset](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page), specifically the "Yellow Taxi Trip Records" from january, february 2022.

**Author**: Hugo Valenzuela

# Building a linear regression model to predict taxi rides duration

## Libraries required